In [ ]:
!pip install fastapi
!pip install --force-reinstall -v "numpy==1.25.2"
!pip install pyLDAvis
!pip install -U gensim

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *
stemmer = SnowballStemmer('russian')

from pprint import pprint

from gensim import corpora, models
from gensim.utils import simple_preprocess

from random import choice
import numpy as np
np.random.seed(1234)

import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 54.1 MB/s eta 0:00:00
  Attempting 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.1.3 which is incompatible.


AttributeError: ignored

Since the corpus I found was a zipped jsonl, there was some googling I needed to do, but here is how I unpacked it at the end:

In [ ]:
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile
zipurl = "https://ngodata.ru/dataset/353ffd68-e787-4ad7-9e21-fef7f31875bf/resource/c747512f-b4fb-4c67-b2f4-1bbedade28c5/download/fedgovnews_final.zip"
with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall() #345MB

import json
with open("/content/fedgovnews_final.jsonl") as jsonl:
  all_articles = [json.loads(line)["text"].replace("\n", " ") for line in jsonl]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def lemmatize_stemming(text: str):
  return stemmer.stem(text)

def preprocess(text: list[str]):
  result = [lemmatize_stemming(token)
            for token in simple_preprocess(text)
            if token not in russian_stopwords and len(token) > 3]
  return result

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
processed_docs = list(map(preprocess, all_articles))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
dictionary = corpora.Dictionary(processed_docs)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
filter_tokens_if_container_documents_are_less_than = 15
filter_tokens_if_appeared_percentage_more_than = 0.5
keep_the_first_n_tokens = 100_000

num_of_topics = 10

dictionary.filter_extremes(
    no_below=filter_tokens_if_container_documents_are_less_than,
    no_above=filter_tokens_if_appeared_percentage_more_than,
    keep_n=keep_the_first_n_tokens)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
tfidf = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
lda_model_tfidf = models.LdaMulticore(tfidf_corpus,
                                      num_topics=num_of_topics,
                                      id2word=dictionary,
                                      passes=10,
                                      workers=4)
for idx, topic in lda_model_tfidf.print_topics(num_of_topics):
    print(f'Topic: {idx} \t Word: {topic}')

print('\nPerplexity TFIDF: ', lda_model_tfidf.log_perplexity(bow_corpus))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic: 0 	 Word: 0.010*"федерац" + 0.009*"закон" + 0.008*"внесен" + 0.007*"федеральн" + 0.005*"изменен" + 0.005*"законопроект" + 0.004*"одобр" + 0.004*"установлен" + 0.004*"государствен" + 0.004*"бюджет"
Topic: 1 	 Word: 0.003*"рубл" + 0.003*"бюджет" + 0.003*"млрд" + 0.002*"совещан" + 0.002*"регион" + 0.002*"программ" + 0.002*"мишустин" + 0.002*"миха" + 0.002*"бюджетн" + 0.002*"рынк"
Topic: 2 	 Word: 0.003*"наш" + 0.003*"медвед" + 0.003*"очен" + 0.002*"стран" + 0.002*"сотрудничеств" + 0.002*"премьер" + 0.002*"говор" + 0.002*"так" + 0.002*"сам" + 0.002*"встреч"
Topic: 3 	 Word: 0.071*"рационализац" + 0.058*"пройдет" + 0.035*"viii" + 0.035*"кубок" + 0.035*"кубк" + 0.035*"worldskills" + 0.034*"екатеринбург" + 0.034*"белоус" + 0.031*"чемпионат" + 0.030*"професс"
Topic: 4 	 Word: 0.046*"празднован" + 0.024*"невск" + 0.024*"княз" + 0.024*"приуроч" + 0.023*"чернышенк" + 0.021*"рожден" + 0.020*"побед" + 0.018*"состо" + 0.017*"лет" + 0.016*"включа"

Perplexity TFIDF:  -7.793684511907093


In [ ]:
test_doc = choice(range(len(processed_docs)))
print(processed_docs[test_doc])
for index, score in sorted(lda_model_tfidf[tfidf_corpus[test_doc]], key=lambda tup: -1*tup[1]):
    print("Topic match score: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, num_of_topics)))

['миха', 'мишустин', 'присутствова', 'церемон', 'открыт', 'реконструкц', 'национальн', 'картин', 'галер', 'афин', 'открыт', 'обновлен', 'здан', 'музе', 'проход', 'рамк', 'торжеств', 'случа', 'лет', 'нача', 'греческ', 'национальн', 'освободительн', 'революц']
Topic match score: 0.8506984114646912	 
Topic: 0.003*"рубл" + 0.003*"бюджет" + 0.003*"млрд" + 0.002*"совещан" + 0.002*"регион"
Topic match score: 0.03791631758213043	 
Topic: 0.071*"рационализац" + 0.058*"пройдет" + 0.035*"viii" + 0.035*"кубок" + 0.035*"кубк"
Topic match score: 0.037588104605674744	 
Topic: 0.046*"празднован" + 0.024*"невск" + 0.024*"княз" + 0.024*"приуроч" + 0.023*"чернышенк"
Topic match score: 0.03696074336767197	 
Topic: 0.003*"наш" + 0.003*"медвед" + 0.003*"очен" + 0.002*"стран" + 0.002*"сотрудничеств"
Topic match score: 0.03683636337518692	 
Topic: 0.010*"федерац" + 0.009*"закон" + 0.008*"внесен" + 0.007*"федеральн" + 0.005*"изменен"


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
tfidf_lda_data = gensimvis.prepare(lda_model_tfidf, tfidf_corpus, dictionary)

pyLDAvis.display(tfidf_lda_data)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
